# naive word2vec
Attack problem by: word2vec 內建的各種 functions  
其中表現最好的在 `sample dataset` 中正確率為 `50%`(比亂猜好三倍)

In [1]:
from gensim.models import word2vec
import numpy as np
from scipy import spatial

# Import & Init jieba
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re

Using TensorFlow backend.
Building prefix dict from /home/sunset/word_contest/datas/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u849ecfdca27003d306f39ca004b82b5b.cache
Loading model cost 1.147 seconds.
Prefix dict has been built succesfully.


### Load datasets

In [2]:
sample = pd.read_csv('datas/sample_test_data.txt')
sample

,id,dialogue,options,answer
0,0,A:你這麼快就知道了,B:全家就是你家\tB:付出不是浪費時間\tB:願意為社會付出的人太少了\tB:我都是一個人...,4
1,1,A:每一支冰塊都不同 水質不同 硬度不同,A:紋路不同\tA:也是你唯一發洩情緒的辦法吧\tA:還記不記得那次你在我家巷口\tA:像拼...,0
2,2,A:這樣沖這麼一大塊網子 要多久時間,B:有更舒適的環境\tB:感覺是一隻很貼心的貓咪\tB:我相信一定可以成功的啦\tB:可以講...,5
3,3,A:兒子啊 都幾點了 你還不睡,B:沒關係啦 我來幫你\tB:我在背書啦 就快背好了\tB:造成誤會 不是很可惜嗎\tB:女...,1
4,4,A:孩子還在念書的時候 看到其他小朋友都有父母 而自己沒有媽媽,A:那孩子就是突然\tA:就是說\tA:禮拜一到學校\tA:媽媽還不會開車\tA:就會問我媽...,4
5,5,A:那牠吐絲這三天三夜\tA:你們不就都要一直在這邊守護著牠囉,B:涂先生\tB:那這些蠶寶寶在這邊吐絲\tB:要吐多久啊\tB:溫度三十多度的話\tB:對...,4
6,6,A:奇怪了 濂僑跑到哪兒去了呢 濂僑 你怎麼啦,B:我可能吃壞肚子了 肚子好痛喔\tB:另外一種比法是 一星期有七天\tB:為什麼不可以 同...,0
7,7,A:是什麼讓玉屏的改變這麼大啊,B:當然是紀老師的魔法囉\tB:都交給這位大力士吧\tB:粗重的交給我就對了\tB:我是八年...,0
8,8,A:曉書啊 你手上拿的 該不是最新的遊戲機吧,B:你不要小看這個扯鈴\t B:互相幫忙一下嘛\tB:一個週末出去走走 沒有關係的啦\tB:...,3
9,9,A:失戀的確會讓人家很難過,A:所以你要縮短認真難過的時間\tA:他走了十三個年頭\tA:獲選之後當然是很開心囉\tA:...,0


In [3]:
x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]
y = sample.answer.values
assert(np.sum([len(_)!=6 for _ in x2]) == 0)

In [4]:
test_datas = pd.read_csv('datas/AIFirstProblem.txt')
test_datas

,id,dialogue,options
0,1,A:媽給你送錢包來啦 來 你看一下是不是這個\tB:對 就是這個 你在哪裡找到它的,A:你看 這是我新買的錢包\tA:我的錢包不見了啦\tA:以後上網咖的錢包在我身上\tA:什...
1,2,A:古人說三日不讀書 面目可憎 我覺得我最近可能臉色太難看了,A:所以想回復我昔日面貌姣好的樣子\tA:是不是要定期來舉辦\tA:各辦理一次才對\tA:能...
2,3,A:你說我們做父母的最擔心的就是這個,A:我剛剛聽你媽說你要讀什麼科\tA:其他老師又集體叛變\tA:聽起來好好玩天啊\tA:只是...
3,4,A:是不是因為以前你的書包太重了 所以你想要發明這個,B:這個應該也是其中之一的原因\tB:好神奇喔\tB:它怎麼突然會這樣\tB:那我去探索一下...
4,5,A:那可不見得 有錢能使鬼推磨,B:好 這個就是我們今天比較級的用法\tB:真的嗎 什麼禮物\tB:有餅乾吃我也要吃\tB:...
5,6,A:真的嗎\tB:當然是真的囉 如果你不相信的話 可以問姜老師啊,C:老師常常看了都很感動\tC:你不要太在意同學們怎麼想\tC:老師先給你們一個功課\tC:...
6,7,A:你今天是要介紹哪一位人物 讓我們認識呢\tB:是和戲劇有關喔 特別是默劇,A:我們不是當事人，不太方便回答相關問題\tA:兩人得到了法國航空提供的巴黎來回機票\tA:...
7,8,A:輝哥 全部都被你吃了 你很過分\tB:就兩顆 你還剩下這樣子 那我吃什麼\tC:沒有 因...,C:然後煮一煮就變成紅色的嗎\tC:我就把它吃完了\tC:現在要把那個木頭給粉碎掉\tC:因...
8,9,A:大嬸啊 我不是說過了嗎 我們村子都是善男信女 老弱婦孺 怎麼會是練功的料呢,B:看來大家對男生女生應該有什麼樣子 還是有很刻板的印象喔\tB:我們讀不讀書關你屁事啊 你...
9,10,A:你只要拿得到滅火器 就代表你的逃生動線是安全的 你就算火滅不掉,A:防災達人接下來要突擊的\tA:是位在桃園八德市的一家雜貨店\tA:之前台語歌曲天后張秀卿...


In [5]:
test_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.dialogue.values]
test_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.options.values]
assert(np.sum([len(_)!=6 for _ in test_x2]) == 0)

### word2vec model

In [10]:
word2vec_model = word2vec.Word2Vec.load('models/word2vec_250.model.bin')
len(word2vec_model.wv.vocab)

614202

In [11]:
word2vec_model.wv.index2word[10]  # given index, find word string in word2vec_model's vocab

'小行星'

In [12]:
word2vec_model.wv.vocab['國王'].index  # given word string, find index in word2vec_model's vocab

772

In [13]:
# Count total occurance to estimate word probability
total_word_cnt = np.sum([_.count for _ in word2vec_model.wv.vocab.values()])
total_word_cnt

176304043

### Naive trial - centroid

In [14]:
def centroid(sentence):
    return np.mean(
        [word2vec_model.wv.word_vec(word) for word in sentence if word in word2vec_model.wv.vocab],
        axis=0
    )

In [17]:
correct = 0
for a, b, ans in zip(x1, x2, y):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]
    
    a_center = centroid(a_sentence)
    b_centers = [centroid(s) for s in b_sentences]
    
    score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
    if np.argmin(score) == ans:
        correct += 1

print('%20s: %3d / %3d' % ('centroid', correct, len(y)))

            centroid:  24 /  50


### Naive trial - weighted centroid

In [18]:
def weighted_centroid(sentence, a=1e-4):
    _ = [a / (a + word2vec_model.wv.vocab[word].count / total_word_cnt) * word2vec_model.wv.word_vec(word)
            for word in sentence if word in word2vec_model.wv.vocab]
    return np.mean(_, axis=0) if len(_) > 0 else np.zeros(word2vec_model.vector_size)

In [20]:
correct = 0
for a, b, ans in zip(x1, x2, y):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]

    a_center = weighted_centroid(a_sentence)
    b_centers = [weighted_centroid(s) for s in b_sentences]

    score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
    if np.argmin(score) == ans:
        correct += 1

print('%20s: %3d / %3d' % ('centroid', correct, len(y)))

            centroid:  30 /  50


### Naive trial - setence embedding

In [14]:
def setence_embedding(sentence, u, a=1e-4):
    vs = weighted_centroid(sentence, a)
    return vs - u * (vs @ u)

In [15]:
u = np.load('datas/sentence_embedding/first_principle_component.npy')

In [16]:
correct = 0
for a, b, ans in zip(x1, x2, y):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]

    a_center = setence_embedding(a_sentence, u)
    b_centers = [setence_embedding(s, u) for s in b_sentences]

    score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
    if np.argmin(score) == ans:
        correct += 1

print('%20s: %3d / %3d' % ('centroid', correct, len(y)))

            centroid:  29 /  50


### Naive trial - cosine similarity between two sets of words

In [17]:
def cos_sim(a_words_set, b_words_set):
    a = [word for word in a_words_set if word in word2vec_model.wv.vocab]
    b = [word for word in b_words_set if word in word2vec_model.wv.vocab]
    if len(a) == 0 or len(b) == 0:
        return 0
    return word2vec_model.wv.n_similarity(a, b)

In [18]:
correct = 0
for a, b, ans in zip(x1, x2, y):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]
    
    score = [cos_sim(a_sentence, bs) for bs in b_sentences]
    if np.argmax(score) == ans:
        correct += 1

print('%20s: %3d / %3d' % ('cos similarity', correct, len(y)))

      cos similarity:  24 /  50


### Naive trial - Word Mover's Distance
- [A Linear Time Histogram Metric for Improved SIFT Matching](http://www.cs.huji.ac.il/~werman/Papers/ECCV2008.pdf)
- [Fast and Robust Earth Mover’s Distances](http://www.cs.huji.ac.il/~werman/Papers/ICCV2009.pdf)
- [From Word Embeddings To Document Distances](http://proceedings.mlr.press/v37/kusnerb15.html)

In [19]:
correct = 0
for a, b, ans in zip(x1, x2, y):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]
    
    score = [word2vec_model.wv.wmdistance(a_sentence, bs) for bs in b_sentences]
    if np.argmin(score) == ans:
        correct += 1

print('%20s: %3d / %3d' % ('document dis', correct, len(y)))

        document dis:  25 /  50


### Naive trial - center word similarity

In [20]:
def center_sim(a, b):
    ap = word2vec_model.predict_output_word(a, topn=2)
    bp = word2vec_model.predict_output_word(b, topn=2)
    mean_sim = 0
    for aw in ap:
        for bw in bp:
            mean_sim += word2vec_model.wv.similarity(aw[0], bw[0]) * aw[1] * bw[1]
    return mean_sim

In [ ]:
correct = 0
for a, b, ans in zip(x1, x2, y):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]
    
    score = [center_sim(a_sentence, bs) for bs in b_sentences]
    if np.argmax(score) == ans:
        correct += 1

print('%20s: %3d / %3d' % ('center word sim', correct, len(y)))

## Output answer on testing datas

In [ ]:
my_ans = []
for a, b in zip(test_x1, test_x2):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]

    a_center = weighted_centroid(a_sentence)
    b_centers = [weighted_centroid(s) for s in b_sentences]

    score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
    my_ans.append(np.argmin(score))

In [ ]:
for i in range(len(test_x1)):
    print('questions:', test_x1[i])
    for j, option in enumerate(test_x2[i]):
        print('\t%d) %s' % (j, option))
    print('my answer:', my_ans[i])
    print('=' * 100)

In [ ]:
with open('answer/attack-naive-word2vec.txt', 'w') as fo:
    fo.write('id,ans\n')
    fo.write('\n'.join(['%d,%s' % (i+1, ans) for i, ans in enumerate(my_ans)]))
    fo.write('\n')

In [ ]:
backup_ans = pd.read_csv('answer/attack-naive-word2vec.backup.txt')
backup_ans = backup_ans.ans.values

In [ ]:
for i in range(len(test_x1)):
    if my_ans[i] == backup_ans[i]:
        continue
    print('questions:', test_x1[i])
    for j, option in enumerate(test_x2[i]):
        print('\t%d) %s' % (j, option))
    print('now    answer:', my_ans[i])
    print('backup answer:', backup_ans[i])
    print('=' * 100)

## Output answer for bagging

In [27]:
a_lst = [1e-3, 9e-4, 8e-4, 7e-4, 6e-4, 5e-4, 5e-4, 3e-4, 2e-4, 1e-4, 9e-5, 8e-5, 7e-5, 6e-5, 5e-5]

In [ ]:
sample_ans_lst = []
sample_ans_lst.append([])
for a, b, ans in zip(x1, x2, y):
    a_sentence = [word for s in a for word in jieba.cut(s)]
    b_sentences = [[word for word in jieba.cut(s)] for s in b]

    a_center = weighted_centroid(a_sentence)
    b_centers = [weighted_centroid(s) for s in b_sentences]

    score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
    sample_ans_lst[-1].append(np.argmin(score))

for alpha in a_lst:
    sample_ans_lst.append([])
    for a, b, ans in zip(x1, x2, y):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        a_center = setence_embedding(a_sentence, u)
        b_centers = [setence_embedding(s, u) for s in b_sentences]

        score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
        sample_ans_lst[-1].append(np.argmin(score))
        
sample_ans_lst = np.array(sample_ans_lst)

In [ ]:
correct = 0
for i in range(len(y)):
    if np.argmax(np.bincount(sample_ans_lst[:, i])) == y[i]:
        correct += 1

print('%20s: %3d / %3d' % ('centroid', correct, len(y)))